In [1]:
import os
import sys
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_theme()

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.strategy.model import Model
from src.strategy.environment import Environment
from src.strategy.agent import Agent
from src.strategy.buffer import Buffer
from src.utils import get_config, read_file
config = get_config.read_yaml()

SyntaxError: invalid syntax (agent.py, line 79)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = len(config['data']['symbols']) * config['data']['num_features']
gamma = config['hyperparameters']['gamma']
gae_lambda = config['hyperparameters']['gae_lambda']
clip_epsilon = config['hyperparameters']['clip_epsilon']
value_loss_coef = config['hyperparameters']['value_loss_coef']
entropy_loss_coef = config['hyperparameters']['entropy_loss_coef']
batch_size = config['hyperparameters']['batch_size']
epochs = config['hyperparameters']['num_epochs']
lr = config['hyperparameters']['learning_rate']
seq_len = config['hyperparameters']['seq_len']
rollout_steps = config['hyperparameters']['rollout_steps']
print(f'input_dim: {input_dim}')
print(f'gamma: {gamma}')
print(f'gae_lambda: {gae_lambda}')
print(f'clip_epsilon: {clip_epsilon}')
print(f'value_loss_coef: {value_loss_coef}')
print(f'entropy_loss_coef: {entropy_loss_coef}')
print(f'batch_size: {batch_size}')
print(f'epochs: {epochs}')
print(f'lr: {lr}')
print(f'seq_len: {seq_len}')
print(f'rollout_steps: {rollout_steps}')
print(f'device: {device}')

In [ ]:
data = read_file.read_merged_training_data()
data

In [ ]:
model = Model(input_dim).to(device)
model

In [ ]:
h_0, c_0 = model.init_hidden_state(batch_size, device)
h_0.shape

In [ ]:
agent = Agent(model,
              device=device,
              learning_rate=lr,
              gamma=gamma,
              gae_lambda=gae_lambda,
              clip_epsilon=clip_epsilon,
              epochs=epochs,
              batch_size=batch_size,
              sequence_length=seq_len,
              value_loss_coef=value_loss_coef,
              entropy_loss_coef=entropy_loss_coef,
              )
agent

In [ ]:
env = Environment(data)
env

In [ ]:
buffer = Buffer(rollout_steps)
buffer

In [ ]:
total_timesteps = 1_000_000
global_timestep_counter = 0
update_counter = 0

In [ ]:
while global_timestep_counter < total_timesteps:
    pass